In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# Ensure correct version
print(f"Using OSMnx version: {ox.__version__}")

# List of cities to analyze
cities = [
    "Amsterdam, Netherlands",
    "Barcelona, Spain",
    "Boston, USA",
    "Berlin, Germany",
    "New York City, USA",
    "Paris, France"
]

# Function to compute street network metrics
def compute_street_metrics(city_name):
    print(f"Processing: {city_name}...")

    try:
        # Load street network (drivable roads only)
        G = ox.graph_from_place(city_name, network_type="drive", simplify=True)

        # Convert MultiDiGraph to undirected MultiGraph
        G_undirected = G.to_undirected()

        # Compute basic statistics
        stats = ox.stats.basic_stats(G_undirected)

        # Compute graph-based centrality measures (using largest connected component)
        largest_cc = max(nx.connected_components(G_undirected), key=len)
        G_lcc = G_undirected.subgraph(largest_cc)

        betweenness = nx.betweenness_centrality(G_lcc)
        closeness = nx.closeness_centrality(G_lcc)

        # Compute circuity (average edge straightness)
        circuity = stats.get("circuity_avg", None)

        # Store results in a dictionary
        return {
            "city": city_name,
            "intersection_density": stats.get("intersection_density_km", None),
            "street_density": stats.get("street_density_km", None),
            "betweenness_avg": sum(betweenness.values()) / len(betweenness) if betweenness else None,
            "closeness_avg": sum(closeness.values()) / len(closeness) if closeness else None,
            "circuity": circuity,
        }
    except Exception as e:
        print(f"Error processing {city_name}: {e}")
        return None

# Compute metrics for all cities
city_metrics = [compute_street_metrics(city) for city in cities]

# Convert to DataFrame and save results
df = pd.DataFrame(city_metrics).dropna()
df.to_csv("city_street_metrics.csv", index=False)

print("\nComparison of city street network metrics:")
print(df)


Using OSMnx version: 2.0.1
Processing: Amsterdam, Netherlands...


In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np
from scipy.stats import entropy

# Ensure correct version
print(f"Using OSMnx version: {ox.__version__}")

# List of cities to analyze
cities = [
    "Amsterdam, Netherlands",
    "Barcelona, Spain",
    "Boston, USA",
    "Berlin, Germany",
    "New York City, USA",
    "Paris, France"
]

# Function to compute street network metrics
def compute_street_metrics(city_name):
    print(f"Processing: {city_name}...")

    try:
        # Load street network (drivable roads only)
        G = ox.graph_from_place(city_name, network_type="drive", simplify=True)

        # Convert MultiDiGraph to undirected MultiGraph (OSMnx 2.0+ compatible)
        G_undirected = ox.utils_graph.get_undirected(G)

        # Compute basic statistics
        stats = ox.stats.basic_stats(G_undirected)

        # Compute graph-based centrality measures (using largest connected component)
        largest_cc = max(nx.connected_components(G_undirected), key=len)
        G_lcc = G_undirected.subgraph(largest_cc)

        betweenness = nx.betweenness_centrality(G_lcc)
        closeness = nx.closeness_centrality(G_lcc)

        # Compute circuity (average edge straightness)
        circuity = stats.get("circuity_avg", None)

        # Compute missing metrics
        avg_degree = average_node_degree(G_undirected)
        pagerank = compute_pagerank(G_lcc)
        avg_street_len = average_street_length(G_undirected)
        street_orientation = street_orientation_analysis(G_undirected)

        # Store results in a dictionary
        return {
            "city": city_name,
            "intersection_density": stats.get("intersection_density_km", None),
            "street_density": stats.get("street_density_km", None),
            "betweenness_avg": sum(betweenness.values()) / len(betweenness) if betweenness else None,
            "closeness_avg": sum(closeness.values()) / len(closeness) if closeness else None,
            "circuity": circuity,
            "avg_node_degree": avg_degree,
            "pagerank": pagerank,
            "avg_street_length": avg_street_len,
            "street_orientation_entropy": street_orientation,
        }
    except Exception as e:
        print(f"Error processing {city_name}: {e}")
        return None


# Function to compute Average Node Degree
def average_node_degree(G):
    degrees = [val for node, val in G.degree()]
    return sum(degrees) / len(degrees)


# Function to compute PageRank
def compute_pagerank(G):
    pagerank = nx.pagerank(G)
    return sum(pagerank.values()) / len(pagerank)


# Function to compute Average Street Length
def average_street_length(G):
    total_length = sum(ox.utils_graph.get_edge_attributes(G, "length").values())
    return total_length / len(G.edges())


# Function to compute Street Orientation Analysis (Entropy)
def street_orientation_analysis(G):
    # Compute angles between all street segments (edges)
    angles = []
    for u, v, data in G.edges(data=True):
        if "geometry" in data:
            angle = np.degrees(np.arctan2(data["geometry"].coords[1][1] - data["geometry"].coords[0][1], 
                                          data["geometry"].coords[1][0] - data["geometry"].coords[0][0]))
            angles.append(angle)
    
    # Compute entropy of angles (this could give an idea of grid vs organic layout)
    angle_entropy = entropy(angles)  # You'll need an entropy function for this
    return angle_entropy


# Compute metrics for all cities
city_metrics = [compute_street_metrics(city) for city in cities]

# Convert to DataFrame and save results
df = pd.DataFrame(city_metrics).dropna()
df.to_csv("city_street_metrics.csv", index=False)

print("\nComparison of city street network metrics:")
print(df)
